In [1]:
'''
In this code, I try to generate Java code by LSTM
Code is loading Java codes from some sources and combine them.
Than code is made a sequence of 50 keywords to yield 51th item.
I put some println(s) to show intermediate outputs.
Selected training java codes are more core library codes,
so selecting a code written for a real life code will look different.
'''
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
from numpy.random import seed
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.


In [2]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r',encoding="utf-8")
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

def process_file(file_name):
    doc = load_doc(file_name)
    lines = doc.split('\n')
    cleaned = []
    invalids = ["/","*"]
    for i in range(len(lines)):
        orig = lines[i].strip()
        #print(orig,len(orig))    
        if len(orig) > 3  :
            if orig[0] not in invalids :
                cleaned.append(orig.lower())
    return cleaned
    
def process_files(files):
    combined = process_file('D:/data2/java_codes/'+files[0])
    for j in range(1,len(files)):
        print(len(combined))
        combined.extend(  process_file('D:/data2/java_codes/'+files[j]))
    return combined    
    
#just get some random files from Java source codes
all_files = process_files(["Arrays.txt","BigInteger.txt","Character.txt","Collections.txt","Formatter.txt"])
print(len(all_files))

1345
2408
4993
7332
9012


In [3]:
#1st 10 lines
all_files[0:10]

['public class arrays {',
 'private static final int min_array_sort_gran = 1 << 13;',
 'private arrays() {}',
 'static final class naturalorder implements comparator<object> {',
 '@suppresswarnings("unchecked")',
 'public int compare(object first, object second) {',
 'return ((comparable<object>)first).compareto(second);',
 'static final naturalorder instance = new naturalorder();',
 'private static void rangecheck(int arraylength, int fromindex, int toindex) {',
 'if (fromindex > toindex) {']

In [29]:
#seperate ( ) + ,  keywords
corpus = []
for s in all_files:
    splits = s.replace(")"," ) ").replace("("," ( ").replace(";"," ; ").replace(","," , ").strip().split(" ")
    for k in splits:        
        if len(k.strip()) > 0:
            #print(k.strip())
            corpus.extend([k.strip()])
            
print("total words = ",len(corpus))    
#print 1st 10 keywords
' '.join(corpus[:10])


total words 61925


'public class arrays { private static final int min_array_sort_gran ='

In [33]:
tokens = corpus
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))
 
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))


Total Tokens: 61925
Unique Tokens: 5024
Total Sequences: 61874


['public class arrays { private static final int min_array_sort_gran = 1 << 13 ; private arrays ( ) {} static final class naturalorder implements comparator<object> { @suppresswarnings ( "unchecked" ) public int compare ( object first , object second ) { return ( ( comparable<object> ) first ) .compareto ( second',
 'class arrays { private static final int min_array_sort_gran = 1 << 13 ; private arrays ( ) {} static final class naturalorder implements comparator<object> { @suppresswarnings ( "unchecked" ) public int compare ( object first , object second ) { return ( ( comparable<object> ) first ) .compareto ( second )']

In [34]:
# integer encode sequences of words
tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(sequences)
sequences2 = tokenizer.texts_to_sequences(sequences)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [39]:
#print some
i = 0
print("keyword = ",sequences[i])
print(" ")
print("encoded = ",sequences2[i])



keyword =  public class arrays { private static final int min_array_sort_gran = 1 << 13 ; private arrays ( ) {} static final class naturalorder implements comparator<object> { @suppresswarnings ( "unchecked" ) public int compare ( object first , object second ) { return ( ( comparable<object> ) first ) .compareto ( second
 
encoded =  [7, 82, 5017, 5, 19, 9, 14, 10, 111, 6, 25, 113, 1065, 3, 19, 5017, 1, 2, 1064, 9, 14, 82, 1412, 110, 5013, 5, 92, 1, 116, 2, 7, 10, 843, 1, 41, 327, 4, 41, 736, 2, 5, 8, 1, 1, 1411, 2, 327, 2, 1063, 1, 736]


In [40]:
sequences2 = np.array(sequences2)
print("sequences2",sequences2.shape)

sequences2 (61874, 51)


In [41]:
X, y = sequences2[:,:-1], sequences2[:,-1]
print(X.shape)
print(y.shape)

(61874, 50)
(61874,)


In [46]:
# 736   2   3 output sequence is 51,52 and 53 of the sequence
print(X[0:3])
print(y[0:3])

[[   7   82 5017    5   19    9   14   10  111    6   25  113 1065    3
    19 5017    1    2 1064    9   14   82 1412  110 5013    5   92    1
   116    2    7   10  843    1   41  327    4   41  736    2    5    8
     1    1 1411    2  327    2 1063    1]
 [  82 5017    5   19    9   14   10  111    6   25  113 1065    3   19
  5017    1    2 1064    9   14   82 1412  110 5013    5   92    1  116
     2    7   10  843    1   41  327    4   41  736    2    5    8    1
     1 1411    2  327    2 1063    1  736]
 [5017    5   19    9   14   10  111    6   25  113 1065    3   19 5017
     1    2 1064    9   14   82 1412  110 5013    5   92    1  116    2
     7   10  843    1   41  327    4   41  736    2    5    8    1    1
  1411    2  327    2 1063    1  736    2]]
[736   2   3]


In [14]:
from keras.utils import to_categorical

# separate into input and output


y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            251250    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 5025)              507525    
Total params: 909,675
Trainable params: 909,675
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
model.fit(X, y, batch_size=128, epochs=50)

Epoch 1/50
61874/61874 [==============================] - 108s 2ms/step - loss: 5.2875 - accuracy: 0.1091
Epoch 2/50
61874/61874 [==============================] - 102s 2ms/step - loss: 4.3665 - accuracy: 0.2127
Epoch 3/50
61874/61874 [==============================] - 154s 2ms/step - loss: 3.6263 - accuracy: 0.3670
Epoch 4/50
61874/61874 [==============================] - 163s 3ms/step - loss: 3.2513 - accuracy: 0.4247
Epoch 5/50
61874/61874 [==============================] - 184s 3ms/step - loss: 2.9906 - accuracy: 0.4579
Epoch 6/50
61874/61874 [==============================] - 194s 3ms/step - loss: 2.7360 - accuracy: 0.4893
Epoch 7/50
61874/61874 [==============================] - 204s 3ms/step - loss: 2.5168 - accuracy: 0.5180
Epoch 8/50
61874/61874 [==============================] - 201s 3ms/step - loss: 2.3528 - accuracy: 0.5374
Epoch 9/50
61874/61874 [==============================] - 292s 5ms/step - loss: 2.2094 - accuracy: 0.5538
Epoch 10/50
61874/61874 [=====================

In [17]:
from pickle import dump
# save the model to file
#model.save('D:/data2/model_javcodes2.h5')
# save the tokenizer
#dump(tokenizer, open('D:/data2/tokenizer_javcodes2.pkl', 'wb'))

In [47]:


def generate_next(model, tokenizer, seq_length, seed_text):
    reverse = dict([ (v,k) for k,v in tokenizer.word_index.items()])
    encoded = tokenizer.texts_to_sequences([seed_text])
    #get head element
    encoded = encoded[0]
    #pad with maxlen  
    padded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    #predict
    yhat = model.predict_classes(padded, verbose=0)
    return reverse[yhat[0]]

def generate_batch(model, tokenizer, seq_length, seed_text, n_words):
	print("seed_text",seed_text)
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# map predicted word index to word
		out_word = generate_next(model, tokenizer, seq_length,in_text)
		# append to input
		in_text += ' ' + out_word
		#print("in_text",in_text)
		result.append(out_word)
	return ' '.join(result)    

In [48]:
generate_batch(model, tokenizer, seq_length, "public class arrays", 50)

seed_text public class arrays


'( char c ) { return forname0 ( name , initialize , loader ) ; private void getdeclaredclasses ( ) throws securityexception { checkmemberaccess ( member.declared , reflection.getcallerclass ( ) , true ) ; return getconstructor0 ( empty , flags ) ; @callersensitive public field getfield ( string name )'

In [50]:
#sample 1 iteration
in_text = "public class arrays"
encoded = tokenizer.texts_to_sequences([in_text])
print("encoded",encoded)
print("encoded",type(encoded[0]))
encoded = encoded[0]
print("encoded",encoded)
padded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
print("padded",padded.shape)
yhat = model.predict_classes(padded, verbose=0)
print("yhat",yhat)
out_word = ''
for word, index in tokenizer.word_index.items():
	if index == yhat:
		out_word = word
		break
out_word 

encoded [[7, 82, 5017]]
encoded <class 'list'>
encoded [7, 82, 5017]
padded (1, 50)
yhat [1]


'('

In [52]:
generate_batch(model, tokenizer, seq_length, "public class arrays", 50)

seed_text public class arrays


'( char c ) { return forname0 ( name , initialize , loader ) ; private void getdeclaredclasses ( ) throws securityexception { checkmemberaccess ( member.declared , reflection.getcallerclass ( ) , true ) ; return getconstructor0 ( empty , flags ) ; @callersensitive public field getfield ( string name )'

In [53]:
generate_batch(model, tokenizer, seq_length, "static final class", 50)

seed_text static final class


'unmodifiableentrysetspliterator<k , v> s ; final sortedmap<k , v> m ; final spliterator<map.entry<k , v>> s ; checkedqueue ( queue<e> queue , class<e> elementtype , class<t> mutex ) { super ( m , mutex ) ; sm = m ; public comparator<? super k> comparator ( ) { return sm.comparator'

In [54]:
generate_batch(model, tokenizer, seq_length, "public int compare", 50)

seed_text public int compare


'( bigdecimal value , locale l ) throws ioexception stringbuilder sb = new stringbuilder ( ) ; leadingsign ( sb , neg ) ; } else if ( c == conversion.general ) { int prec = ( precision == doubleconsts.min_exponent - 1 ) ; int len = precision ; if'